In [1]:
import tflearn
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn import preprocessing
import chess_game as cg
import time

/Users/pschale/anaconda/envs/tf/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
board = tflearn.layers.core.input_data(shape=(None, 64, 8), name='board_input')
aux = tflearn.layers.core.input_data(shape=(None, 7), name='aux_input')
Y = tflearn.layers.core.input_data(shape=(None, 3), name='labels')

board_input_reshaped = tf.reshape(board, [-1, 8, 8, 8])

conv4 = tflearn.conv_2d(board_input_reshaped, 32, 4, activation='relu')


conv2 = tflearn.conv_2d(board_input_reshaped, 32, 2, activation='relu')
conv2_4 = tflearn.conv_2d(conv2, 64, 4, activation='relu')
# note: this is how to get separate branches of a larger NN and then have them converge
fc1 = tflearn.layers.core.fully_connected(tf.concat([conv4, conv2_4], 3), n_units=1000, activation='relu')

fc2 = tflearn.layers.core.fully_connected(tf.concat([fc1, aux], 1), n_units=2000, activation='relu')

net = tflearn.layers.core.fully_connected(fc2, n_units=3, activation='softmax')

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=net, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.00001).minimize(loss)

init = tf.global_variables_initializer()
#net = tflearn.regression(net, optimizer='adam', metric=loss,
#                         loss='categorical_crossentropy')
#model = tflearn.DNN(net, tensorboard_verbose=0)


Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [3]:
a = pd.read_csv('board_positions/board_positions_1.csv')
a = a[~a.isnull().any(axis=1)]
a = a[a['moves_in_game'] - a['move_num']<10]
le = preprocessing.LabelEncoder()
le.fit(['k', 'q', 'r', 'b', 'n', 'p', ' ', 'P', 'N', 'B', 'R', 'Q', 'K'])

bcols = a.columns[:64]

a[bcols] = le.transform(a[bcols])
    

In [4]:
def get_training_data(data):
    
    d = data.sample(100)
    train = d[d.columns[:71]]
    board = train[train.columns[:64]]
    aux = train[train.columns[64:]]
    board_onehot = np.zeros((100, 64, 8))
    board_onehot[:, :, 0] = board.isin([7, 8, 9, 10, 11, 12])
    board_onehot[:, :, 1] = board.isin([0, 1, 2, 3, 4, 5])
    board_onehot[:, :, 2] = board.isin([0, 12])
    board_onehot[:, :, 3] = board.isin([1, 11])
    board_onehot[:, :, 4] = board.isin([2, 10])
    board_onehot[:, :, 5] = board.isin([3, 9])
    board_onehot[:, :, 6] = board.isin([4, 8])
    board_onehot[:, :, 7] = board.isin([5, 7])
    labels = np.zeros((100,3))
    labels[d['winner']==-1, 0] = 1
    labels[d['winner']==0, 1] = 1
    labels[d['winner']==1, 2] = 1
    
    return board_onehot, aux, labels
    

In [5]:
saver = tf.train.Saver()

sess = tf.Session()
sess.run(init)

for epoch in range(2):  # 2 epochs
    for i in range(200):
        batch_board, batch_aux, batch_ys = get_training_data(a)
        sess.run(optimizer, feed_dict={board: batch_board, aux: batch_aux, Y: batch_ys})
        cost = sess.run(loss, feed_dict={board: batch_board, aux: batch_aux, Y: batch_ys})

        if i % 20 == 0:
            print("Epoch:", '%03d' % (epoch + 1), "Step:", '%03d' % i,
                  "Loss:", str(cost))

Epoch: 001 Step: 000 Loss: 1.0769849
Epoch: 001 Step: 020 Loss: 1.010529
Epoch: 001 Step: 040 Loss: 1.0150131
Epoch: 001 Step: 060 Loss: 0.99164337
Epoch: 001 Step: 080 Loss: 0.99367905
Epoch: 001 Step: 100 Loss: 0.98514915
Epoch: 001 Step: 120 Loss: 0.9774544
Epoch: 001 Step: 140 Loss: 1.0172251
Epoch: 001 Step: 160 Loss: 0.96030855
Epoch: 001 Step: 180 Loss: 0.9511741
Epoch: 002 Step: 000 Loss: 0.9727736
Epoch: 002 Step: 020 Loss: 0.9679552
Epoch: 002 Step: 040 Loss: 0.9913797
Epoch: 002 Step: 060 Loss: 0.9046955
Epoch: 002 Step: 080 Loss: 0.9158521
Epoch: 002 Step: 100 Loss: 0.9149568
Epoch: 002 Step: 120 Loss: 0.90831405
Epoch: 002 Step: 140 Loss: 0.9117123
Epoch: 002 Step: 160 Loss: 0.90922034
Epoch: 002 Step: 180 Loss: 0.85467756


In [6]:
batch_board, batch_aux, batch_ys = get_training_data(a)
sess.run(net, feed_dict={board: batch_board, aux: batch_aux, Y: batch_ys})

array([[6.7206049e-01, 6.8570431e-03, 3.2108244e-01],
       [5.3206545e-01, 5.6696110e-03, 4.6226490e-01],
       [1.3792847e-01, 2.1227296e-04, 8.6185926e-01],
       [7.1345377e-01, 5.9932529e-04, 2.8594694e-01],
       [8.1638682e-01, 3.1134982e-03, 1.8049967e-01],
       [4.5088416e-01, 1.5573296e-03, 5.4755855e-01],
       [3.1909996e-01, 4.5476327e-04, 6.8044531e-01],
       [5.5451572e-01, 5.5723463e-04, 4.4492704e-01],
       [5.8062738e-01, 1.3117355e-03, 4.1806096e-01],
       [5.7681906e-01, 4.8390054e-04, 4.2269704e-01],
       [3.5568425e-01, 3.8733560e-04, 6.4392841e-01],
       [6.6698152e-01, 8.1793760e-04, 3.3220059e-01],
       [5.4320943e-01, 1.9756102e-03, 4.5481497e-01],
       [5.6778640e-01, 1.5958974e-03, 4.3061772e-01],
       [1.2550253e-01, 3.4418155e-04, 8.7415332e-01],
       [9.9495433e-02, 2.0787208e-03, 8.9842588e-01],
       [4.8005071e-01, 1.9445553e-03, 5.1800466e-01],
       [9.2297308e-02, 5.1655236e-04, 9.0718615e-01],
       [7.4250311e-01, 1.164

In [7]:
saver.save(sess, '/Users/pschale/pythonstuff/chess_ai_project/test_saved_CNN')

Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'
Type is unsupported, or the types of the items don't match field type in CollectionDef.
'NoneType' object has no attribute 'name'


'/Users/pschale/pythonstuff/chess_ai_project/test_saved_CNN'

In [8]:
a = cg.game_board()
next_inputs = [ele.get_NN_inputs() for ele in a.find_all_next_board_positions()]
batch_board = np.array([ele[0] for ele in next_inputs])
batch_aux = np.array([ele[1] for ele in next_inputs])
sess.run(net, feed_dict={board: batch_board, aux: batch_aux})

array([[0.66404384, 0.00343669, 0.33251947],
       [0.6768603 , 0.00337868, 0.31976116],
       [0.7156529 , 0.00308562, 0.28126144],
       [0.68724227, 0.00331419, 0.30944356],
       [0.6775251 , 0.00343225, 0.31904262],
       [0.69907445, 0.00323504, 0.29769057],
       [0.66996163, 0.00322153, 0.32681683],
       [0.6801045 , 0.00318068, 0.31671485],
       [0.6916137 , 0.00314461, 0.3052417 ],
       [0.6899245 , 0.00309886, 0.30697668],
       [0.7165306 , 0.00309808, 0.2803713 ],
       [0.6950692 , 0.00310822, 0.30182266],
       [0.6887055 , 0.00322423, 0.3080703 ],
       [0.68773323, 0.00308759, 0.30917922],
       [0.6972107 , 0.00319519, 0.29959407],
       [0.7337678 , 0.00294746, 0.2632848 ],
       [0.68326306, 0.00318891, 0.31354806],
       [0.7039133 , 0.00298165, 0.29310504],
       [0.65740556, 0.00333912, 0.33925533],
       [0.6639371 , 0.00326991, 0.33279303]], dtype=float32)

In [ ]:
tf.saved_model.simple_save(sess, 
                            '/Users/pschale/pythonstuff/chess_ai_project/simple_saved',
                          inputs={"board":board, "aux":aux},
                          outputs={"net": net})

In [ ]:
b = cg.game_board()
next_positions = b.find_all_next_board_positions('W')
next_positions_ar = np.array([ele.to_csv_format() for ele in next_positions])
next_positions_ar[:, :64] = le.transform(next_positions_ar[:, :64])


In [ ]:
next_positions = b.find_all_next_board_positions('W')
next_positions_ar = np.array([ele.to_csv_format() for ele in next_positions])
next_positions_ar[:, :64] = le.transform(next_positions_ar[:, :64])

next_scores = sess.run(net, feed_dict={X:next_positions_ar})
b = next_positions[np.argmax(next_scores[:,2])]

In [ ]:
print(b)

In [ ]:
b.move('Qxe5')

In [ ]:
print(b)

In [ ]:
import importlib
importlib.reload(cg)

In [ ]:
a = pd.read_csv('board_positions/board_positions_1.txt')

In [ ]:
a[a['moves_in_game'] - a['move_num']<10][a.columns[:71]]

In [ ]:
tf.__version__

In [ ]:
a = pd.read_csv('board_positions/board_positions_1.csv')
a = a[~a.isnull().any(axis=1)]
a = a[a['moves_in_game'] - a['move_num']<10]
le = preprocessing.LabelEncoder()
le.fit(['k', 'q', 'r', 'b', 'n', 'p', ' ', 'P', 'N', 'B', 'R', 'Q', 'K'])

bcols = a.columns[:64]

a[bcols] = le.transform(a[bcols])

In [ ]:
c = a.sample(100)

In [ ]:
c.shape

In [ ]:
c.as_matrix()[:, :64]

In [ ]:
start = time.time()
train = c[c.columns[:71]]
train_onehot = np.zeros((100, 71, 8))
train_onehot[:, :, 0] = train.isin([7, 8, 9, 10, 11, 12])
train_onehot[:, :, 1] = train.isin([0, 1, 2, 3, 4, 5])
train_onehot[:, :, 2] = train.isin([0, 12])
train_onehot[:, :, 3] = train.isin([1, 11])
train_onehot[:, :, 4] = train.isin([2, 10])
train_onehot[:, :, 5] = train.isin([3, 9])
train_onehot[:, :, 6] = train.isin([4, 8])
train_onehot[:, :, 7] = train.isin([5, 7])
end = time.time()
print(end - start)

In [ ]:
train_onehot[:, :, 0]